In [ ]:
cd /content/drive/MyDrive/Thesis

/content/drive/MyDrive/Thesis


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 45 kB 1.8 MB/s 
     |████████████████████████████████| 311 kB 7.0 MB/s 
     |████████████████████████████████| 3.5 MB 76.0 MB/s 
     |████████████████████████████████| 243 kB 81.6 MB/s 
     |████████████████████████████████| 133 kB 81.1 MB/s 
     |████████████████████████████████| 1.1 MB 67.8 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 312 kB 93.0 MB/s 
     |████████████████████████████████| 311 kB 99.0 MB/s 
     |████████████████████████████████| 311 kB 96.1 MB/s 
     |████████████████████████████████| 306 kB 83.4 MB/s 
     |████████████████████████████████| 298 kB 82.4 MB/s 
     |████████████████████████████████| 298 kB 52.6 MB/s 
     |████████████████████████████████| 290 kB 75.8 MB/s 
     |████████████████████████████████| 290 kB 50.4 MB/s 
     |████████████████████████████████| 56 kB 6.4 MB/s 
     |████████████████████████████████| 290 kB 105.2 MB/s 
     |██████████████

In [ ]:
model_checkpoint = 'facebook/bart-large-cnn'


In [ ]:
!pip install tqdm -U


     |████████████████████████████████| 76 kB 2.9 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [ ]:
from datasets import load_metric


In [ ]:
from transformers import AutoTokenizer
    
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Thesis/data/1024_length_data/train_pairs.csv", index_col=0)
df

,title,abstract,title_length,abstract_length
0,Natural Image Bases to Represent Neuroimaging ...,Visual inspection of neuroimagery is susceptib...,7,132
1,Sluice Resolution without Hand-Crafted Feature...,Sluice resolution in English is the problem of...,9,110
2,Learning Translation Models from Monolingual C...,Translation models often fail to generate good...,7,152
3,Sentiment Adaptive End-to-End Dialog Systems,End-to-end learning framework is useful for bu...,5,119
4,User-Friendly Text Prediction For Translators,Text prediction is a form of interactive machi...,5,134
...,...,...,...,...
21437,"Arabic Tokenization, Part-of-Speech Tagging an...",We present an approach to using a morphologica...,11,58
21438,Using Semantically Motivated Estimates to Help...,Research into the automatic acquisition of sub...,8,530
21439,A Mathematical Exploration of Why Language Mod...,"Autoregressive language models, pretrained usi...",11,194
21440,Do You Know That Florence Is Packed with Visit...,"When a speaker, Mary, asks ""Do you know that F...",15,174


In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.to_csv("/content/drive/MyDrive/Thesis/data/1024_length_data/1024_characters_pairs.csv")

In [ ]:
df = df.drop(columns=["title_length", "abstract_length", "token_len"])

In [ ]:
df_train = df[:11864]
df_valid = df[11864:]

In [ ]:
from datasets import Dataset
from datasets import load_dataset, load_metric
train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)
metric = load_metric("rouge")


In [ ]:
max_input_length = 1024
max_target_length = 512

def preprocess_function(examples):
    model_inputs = tokenizer(examples["abstract"], max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["title"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
preprocess_function(valid_dataset[:2])


{'input_ids': [[0, 1121, 2136, 1472, 2982, 3146, 1023, 9762, 6, 10, 467, 3881, 7, 3094, 5, 1472, 9, 10, 2136, 31, 37617, 1575, 4, 5454, 7926, 7, 745, 10, 239, 12, 12955, 2136, 1472, 2982, 3146, 1023, 9762, 467, 680, 5, 9600, 9, 27963, 414, 13, 42, 3685, 8, 9, 15924, 2051, 12, 6504, 7153, 1472, 41517, 4, 1216, 743, 10114, 5647, 31, 5, 754, 14, 5, 3685, 16, 145, 3032, 11, 13084, 31, 678, 2939, 9, 6885, 2982, 3146, 1023, 14133, 414, 4, 96, 42, 2225, 6, 52, 1701, 5, 1330, 3685, 9, 2136, 19850, 6, 147, 52, 2813, 7, 3094, 5, 4577, 19850, 9, 10, 2136, 31, 5377, 4, 166, 64, 304, 12980, 2777, 22997, 102, 25, 10, 739, 1787, 9, 8531, 16274, 414, 13, 42, 3685, 4, 166, 1455, 16964, 13, 15582, 5, 2136, 19850, 936, 8, 8085, 10, 1233, 3855, 81, 10, 18043, 467, 4, 166, 172, 311, 14, 5, 2136, 12, 48235, 467, 64, 28, 341, 7, 1477, 819, 15, 10, 30082, 20399, 179, 17985, 1253, 35019, 3685, 8, 64, 4296, 8, 12775, 3349, 4438, 5, 278, 9, 1984, 41762, 13, 10, 2136, 4, 2], [0, 170, 892, 5, 13879, 9, 16854, 11, 

In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)


  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
batch_size = 4
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-lm_al_paper",
    evaluation_strategy = "epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    gradient_accumulation_steps=8,
    save_steps = 500,
    logging_steps = 185,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)





```
Examples:
    >>> predictions = ["hello there", "general kenobi"]
    >>> references = ["hello there", "general kenobi"]
    >>> bertscore = datasets.load_metric("bertscore")
    >>> results = bertscore.compute(predictions=predictions, references=references, lang="en")
    >>> print([round(v, 2) for v in results["f1"]])
    [1.0, 1.0]
  
```



In [ ]:
import nltk
import numpy as np
nltk.download('punkt')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: abstract, title.
***** Running training *****
  Num examples = 11864
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 1110


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,2.501000,2.325624,22.987200,10.613500,18.725200,18.781000,58.411000
1,1.787800,2.124046,26.697200,12.661100,21.403200,21.451600,58.393000
2,1.109200,2.227393,27.254100,12.690100,21.760400,22.084200,58.401000


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: abstract, title.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
Saving model checkpoint to bart-large-cnn-finetuned-lm_al_paper/checkpoint-500
Configuration saved in bart-large-cnn-finetuned-lm_al_paper/checkpoint-500/config.json
Model weights saved in bart-large-cnn-finetuned-lm_al_paper/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bart-large-cnn-finetuned-lm_al_paper/checkpoint-500/tokenizer_config.json
Special tokens file saved in bart-large-cnn-finetuned-lm_al_paper/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: abstract, title.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
Saving model checkpoint to bart-large-cnn-finetuned-lm_al_paper/c

TrainOutput(global_step=1110, training_loss=1.941620615366343, metrics={'train_runtime': 3242.812, 'train_samples_per_second': 10.976, 'train_steps_per_second': 0.342, 'total_flos': 1.4958134766993408e+16, 'train_loss': 1.941620615366343, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./output/bart_large")


Configuration saved in ./output/bart_large/config.json
Model weights saved in ./output/bart_large/pytorch_model.bin


In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("./output/bart_large")

In [ ]:
import pandas as pd

test_samples = pd.read_csv("/content/drive/MyDrive/Thesis/data/1024_length_data/test_pairs.csv", index_col=0)
test_samples

,title,abstract,title_length,abstract_length
0,Learning Latent Semantic Annotations for Groun...,Previous work on grounded language learning di...,11,121
1,Partially Supervised Sense Disambiguation by L...,Supervised and semi-supervised sense disambigu...,13,140
2,Hawkes Processes for Continuous Time Sequence ...,Classification of temporal textual data sequen...,15,68
3,A Unified Single Scan Algorithm for Japanese B...,We describe an algorithm for Japanese analysis...,13,62
4,Generating Coherent Event Schemas at Scale,Chambers and Jurafsky (2009) demonstrated that...,6,127
...,...,...,...,...
5356,Bridging Information-Seeking Human Gaze and Ma...,"In this work, we analyze how human gaze during...",8,118
5357,Quantum-inspired Neural Network for Conversati...,We provide a novel perspective on conversation...,7,116
5358,The BQ Corpus: A Large-scale Domain-specific C...,This paper introduces the Bank Question (BQ) c...,13,174
5359,Doc2hash: Learning Discrete Latent variables f...,Learning to hash via generative model has beco...,8,131


In [ ]:
abstracts = test_samples.abstract.to_list()
titles = test_samples.title.to_list()

In [ ]:
def creat_eval_pairs(model, tokenizer, abstracts, titles):
  preds = []
  for abstract, title in zip(abstracts, titles):
    encoding = tokenizer.encode_plus(abstract, return_tensors = "pt")
    inputs = encoding["input_ids"].to("cuda")
    attention_masks = encoding["attention_mask"].to("cuda")
    title_ids = model.generate(
            input_ids = inputs,
            attention_mask = attention_masks,
            max_length = 30,
            num_beams = 5,
            num_return_sequences = 5,
            repetition_penalty=2.0, 
            length_penalty=15.0,
            early_stopping = True,
            )
    result = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in title_ids]
    s=""
    for t in result:
      s = s + "<TITLE>" + t
    preds.append(s)
    if len(preds) % 500 == 0:
      print("original title: ", title)
      print("generated title: ", preds[-1:])
  return preds, titles

In [ ]:
model.to("cuda")

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
   

In [ ]:
preds, titles = creat_eval_pairs(model, tokenizer, abstracts, titles)

original title:  Paraphrase-Driven Learning for Open Question Answering
generated title:  ['<TITLE>Learning a Semantic Lexicon and Linear Ranking Function for Open-Domain Question Answering over Web Extractions: A Machine<TITLE>Learning a Semantic Lexicon and Linear Ranking Function for Open-Domain Question Answering over Web Extractions with Parallel Per<TITLE>Learning a Semantic Lexicon and Linear Ranking Function for Open-Domain Question Answering over Web Corpora from Web Extract<TITLE>Learning a Semantic Lexicon and Linear Ranking Function for Open-Domain Question Answering over Web Extractions with a Parallel<TITLE>Learning a Semantic Lexicon and Linear Ranking Function for Open-Domain Question Answering over Web Corpora using Parallel Per']
original title:  Robustness and Generalization of Role Sets: PropBank vs. VerbNet
generated title:  ['<TITLE>Robustness and Generalization of Thematic Roles for Semantic Role Labeling: An Empirical Study on the<TITLE>Robustness and Generaliza

In [ ]:
pred_target_pairs = pd.DataFrame(list(zip(preds, titles)), columns=['predictions', 'targets'])

In [ ]:
pred_target_pairs.to_csv("/content/drive/MyDrive/Thesis/output/preds_targets_pairs/bart-large.csv")